In [11]:
import json
import pandas as pd

In [12]:
# Raw reviews data file as received from Yelp
all_reviews_file = '../raw-data/yelp_academic_dataset_review.json'
all_businesses_file = '../raw-data/yelp_academic_dataset_business.json'

In [13]:
def get_df(json_file_name, max_rows=None, city=None, select_keys=None):
    """ Return dataframe from raw data.
    All rows unless max_rows is set. All cities unless city is set. All columns unless select_keys is set.
    """
    with open(json_file_name, 'r') as f:
        i_row = 0
        df_dict_list = []
        for line in f:
            row_dict = json.loads(line)
            row_city = row_dict.get('city', '')
            if select_keys is not None:
                row_dict = {k: row_dict[k] for k in select_keys}
            if (city is None) or (city == row_city):
                df_dict_list.append(row_dict)
                i_row += 1
            if (max_rows is not None) and (i_row >= max_rows):
                break
        df = pd.DataFrame(df_dict_list)
        return df

In [14]:
df = get_df(all_businesses_file, select_keys=['name', 'city', 'state'])
df.head()

,city,name,state
0,Tempe,Innovative Vapors,AZ
1,Las Vegas,Cut and Taste,NV
2,Toronto,Pizza Pizza,ON
3,Oakdale,Plush Salon and Spa,PA
4,Toronto,Comfort Inn,ON


In [15]:
# Print all unique cities from most business listings to least

cities_series = df['city'].value_counts()

print('{} unique city names'.format(len(cities_series)))

for city, n_businesses in zip(cities_series.index, cities_series):
    print('{:50} {:6}'.format(city, n_businesses))

878 unique city names
Las Vegas                                           22892
Toronto                                             14540
Phoenix                                             14468
Scottsdale                                           6917
Charlotte                                            6912
Pittsburgh                                           5275
Montréal                                             4785
Mesa                                                 4714
Henderson                                            3788
Tempe                                                3703
Edinburgh                                            3601
Chandler                                             3325
Cleveland                                            2785
Madison                                              2711
Gilbert                                              2574
Glendale                                             2555
Mississauga                                       

Here's a list of city listings that caught my eye that need to be cleaned up:

N W Las Vegas, Mesa AZ, "Phoenix,", Phoenix AZ, Scottsdale AZ, SCOTTSDALE AZ, Fort  Mill, North Las Vegas, las vegas, N Las Vegas, NORTH YORK, University Hts, Bedford Hts, Bedford HTS, LasVegas, Madison WI, Lake Las Vegas,

In short, it looks like problems with city names include the following:
- Added punctuation ("Phoenix,")
- Additional cardinal directions added to city ("N Las Vegas")
- Inconsistent capitalization
- Added state at end of city name
- Missing space ("LasVegas")
- Inconsistent abbreviation (heights vs hts, township vs twp)

In [16]:
replace_dict = {'township': 'twp', 'heights': 'hts'}
punc_list = ['.', ',', '-']

def format_city(city):
    for key in replace_dict:
        city = city.replace(key, replace_dict[key])
    for punc in punc_list:
        city = city.replace(punc, '')
    city = city.replace(' ', '').lower()
    return city

In [17]:
df['city'] = df['city'].apply(format_city)
df['city'].head()

0       tempe
1    lasvegas
2     toronto
3     oakdale
4     toronto
Name: city, dtype: object

In [18]:
# Print all unique cities from most business listings to least

cities_series = df['city'].value_counts()

print('{} unique city names'.format(len(cities_series)))

for city, n_businesses in zip(cities_series.index, cities_series):
    print('{:50} {:6}'.format(city, n_businesses))

820 unique city names
lasvegas                                            22931
toronto                                             14543
phoenix                                             14469
scottsdale                                           6917
charlotte                                            6913
pittsburgh                                           5275
montréal                                             4785
mesa                                                 4770
henderson                                            3788
tempe                                                3703
edinburgh                                            3601
chandler                                             3325
cleveland                                            2785
madison                                              2711
gilbert                                              2574
glendale                                             2555
mississauga                                       

Simply cleaning up spaces, townships/heights, capitalization, and punctuation reduces the number of unique city names from 878 to 820.

Next let's clean up problems with adding states or directions to the city.

In [20]:
top_30_list = [ format_city(city) for city in cities_series.index ]
state_abbrev_list = ['nv', 'wi', 'az']

def fix_dup_city(city):
    # Get rid of state abbrevs at end of city name
    if city[-2:] in state_abbrev_list:
        print('changing {} to {}'.format(city, city[:-2]))
        city = city[:-2]
    # Change anything with las vegas in it to las vegas. Don't need to differentiate between N Las Vegas and Las Vegas
    if city.find('lasvegas') >= 0:
        if city != 'lasvegas':
            print('changing {} to {}'.format(city, 'lasvegas'))
        city = 'lasvegas'
    return city

df['city'] = df['city'].apply(fix_dup_city)

changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing nlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas to lasvegas
changing northlasvegas t

In [21]:
# Print all unique cities from most business listings to least

cities_series = df['city'].value_counts()

print('{} unique city names'.format(len(cities_series)))

for city, n_businesses in zip(cities_series.index, cities_series):
    print('{:50} {:6}'.format(city, n_businesses))

804 unique city names
lasvegas                                            24123
toronto                                             14543
phoenix                                             14470
scottsdale                                           6919
charlotte                                            6913
pittsburgh                                           5275
montréal                                             4785
mesa                                                 4771
henderson                                            3788
tempe                                                3703
edinburgh                                            3601
chandler                                             3325
cleveland                                            2785
madison                                              2712
gilbert                                              2574
glendale                                             2555
mississauga                                       

That reduced the unique city names from 820 to 804